In [1]:
import gensim
import re
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import numpy as np
from pandarallel import pandarallel
import seaborn as sns


pandarallel.initialize(progress_bar=True, nb_workers=6)
pd.options.display.max_rows = 200


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Document Corpus 

In [2]:
kdf = pd.read_csv("../v1/kawiki.csv")
kdf.text = kdf.text.fillna('')

kdf.head(3)

,id,ts,title,author,text,sha1,isRedirection
0,2,2004-01-26T08:48:14Z,ვიკიპედია:Orphaned articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",ptlg4dsza52o10rnmk59f282y0fptzi,False
1,3,2004-01-26T00:26:51Z,ვიკიპედია:Most wanted articles,204.95.98.251,,pezx7385o3910tfwsxb3suac2dw5g9i,False
2,4,2004-01-26T00:38:52Z,ვიკიპედია:Short articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",84gitu5sgfaec85yfccjzvl2kfafvn1,False


In [4]:
def cleanup(doc):
    doc = str(doc).lower()
    # doc = re.sub(r"[^Ⴀ-ჿⴀ-ⴥᲐ-Ჿ0-9-–—_%a-zA-Z]", ' ', doc)
    doc = re.sub(r"[^Ⴀ-ჿⴀ-ⴥᲐ-Ჿ0-9]", ' ', doc)
    doc = re.sub(r"\s+", ' ', doc)
    return doc


# kdf['cleantext'] = kdf['title'] + '. ' + kdf['text']
kdf['cleantext'] = kdf.text.parallel_apply(cleanup)
kdf.head(3)

,id,ts,title,author,text,sha1,isRedirection,cleantext
0,2,2004-01-26T08:48:14Z,ვიკიპედია:Orphaned articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",ptlg4dsza52o10rnmk59f282y0fptzi,False,
1,3,2004-01-26T00:26:51Z,ვიკიპედია:Most wanted articles,204.95.98.251,,pezx7385o3910tfwsxb3suac2dw5g9i,False,
2,4,2004-01-26T00:38:52Z,ვიკიპედია:Short articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",84gitu5sgfaec85yfccjzvl2kfafvn1,False,11085


In [5]:
# Drop useless wiki articles

# Empty articles
kdf.drop(kdf[(kdf.cleantext == ' ') | (kdf.cleantext == '')].index, inplace=True)
kdf.drop(kdf[kdf.cleantext.apply(lambda x: bool(re.compile("[ა-ჿ]").search(x)))==False].index, inplace=True)

# Mediawiki articles
kdf.drop(kdf[kdf.title.str.startswith("მედიავიკი:")].index, inplace=True)
kdf.drop(kdf[kdf.title.str.startswith("ვიკიპედია:")].index, inplace=True)
kdf.drop(kdf[kdf.title.str.startswith("თარგი:")].index, inplace=True)
kdf.drop(kdf[kdf.title.str.startswith("მოდული:")].index, inplace=True)
kdf.drop(kdf[kdf.title.str.startswith("ფაილი:")].index, inplace=True)

# Redirection
kdf.drop(kdf[kdf.isRedirection].index, inplace=True)

kdf

,id,ts,title,author,text,sha1,isRedirection,cleantext
234,604,2022-04-15T07:50:59Z,ედუარდ შევარდნაძე,Melberg,\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი...,k4tckhizmxggllhid4m0uob0nmwbu0h,False,ედუარდ ამბროსის ძე შევარდნაძე დ 25 იანვარი 19...
236,608,2020-12-01T07:15:40Z,1 იანვარი,Namdvili,\n1 იანვარი — გრიგორიანული კალენდრის პირველი დ...,9hua4ulox6vj429j2nbx3404fylhw5t,False,1 იანვარი გრიგორიანული კალენდრის პირველი დღე ...
237,609,2020-01-18T07:59:48Z,25 იანვარი,95.104.15.233,\n25 იანვარი — გრიგორიანული კალენდრის 25-ე დღე...,nburza2tu9lqypz9cirawwpcg3s7e8z,False,25 იანვარი გრიგორიანული კალენდრის 25 ე დღე წლ...
238,610,2021-11-08T19:32:05Z,1928,Arkaitz1974,\n== მოვლენები ==\n* 11 თებერვალი : შვეიცარიის...,g2uj2d76l6yrbtu1p4wvmnx7vacz64z,False,მოვლენები 11 თებერვალი შვეიცარიის ქალაქ ზანქტ...
282,823,2022-05-09T20:01:56Z,საქართველო,GiorgiXIII,"\n\nსაქართველო () — სახელმწიფო ევრაზიაში, კავკ...",9kmab2upb1nr0ad1ywgr6ldl787kyo3,False,საქართველო სახელმწიფო ევრაზიაში კავკასიაში შა...
...,...,...,...,...,...,...,...,...
320738,532936,2022-05-19T20:24:07Z,კატეგორია:სამხრეთ კორეის საერთაშორისო ხელშეკრუ...,იაკობ მახარაძე,კატეგორია:საერთაშორისო ხელშეკრულებები ქვეყნები...,1mvdnl2ggxg80q12sh5aw4dwfbbt3pu,False,კატეგორია საერთაშორისო ხელშეკრულებები ქვეყნები...
320740,532938,2022-05-20T04:35:44Z,ვახტანგ რობაქიძე,Jaba1977,ვახტანგ რობაქიძე (მოსამართლეობის კანდიდატი) და...,iavvdscaounfaz6sqowhc2ogsvp4u0l,False,ვახტანგ რობაქიძე მოსამართლეობის კანდიდატი დაბ ...
320741,532939,2022-05-20T08:54:31Z,ნინა კარპაჩოვა,Ekkatterrinna,\n\nნინა კარპაჩოვა (უკრ. Ні́на Іва́нівна Карпа...,m955153juz99zg6n3e4pphtna8s6f8t,False,ნინა კარპაჩოვა უკრ დ 12 აგვისტო 1957 მოლდავეთ...
320743,532947,2022-05-20T08:27:24Z,ჰაილბრონის აჩრდილი,Lisztomaniac,\nthumb|right|250px|მემორიალური ნიშნული ოფიცერ...,6sgtfdyggb9svx8whe22owjhwkm8fia,False,250 მემორიალური ნიშნული ოფიცერი კიზევეტერის მ...


In [6]:
kdf['cleantext'] = kdf['title'].apply(cleanup) + ' ' + kdf['cleantext'] + ' ' + kdf['title'].apply(cleanup)
kdf.head()

,id,ts,title,author,text,sha1,isRedirection,cleantext
234,604,2022-04-15T07:50:59Z,ედუარდ შევარდნაძე,Melberg,\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი...,k4tckhizmxggllhid4m0uob0nmwbu0h,False,ედუარდ შევარდნაძე ედუარდ ამბროსის ძე შევარდნა...
236,608,2020-12-01T07:15:40Z,1 იანვარი,Namdvili,\n1 იანვარი — გრიგორიანული კალენდრის პირველი დ...,9hua4ulox6vj429j2nbx3404fylhw5t,False,1 იანვარი 1 იანვარი გრიგორიანული კალენდრის პი...
237,609,2020-01-18T07:59:48Z,25 იანვარი,95.104.15.233,\n25 იანვარი — გრიგორიანული კალენდრის 25-ე დღე...,nburza2tu9lqypz9cirawwpcg3s7e8z,False,25 იანვარი 25 იანვარი გრიგორიანული კალენდრის ...
238,610,2021-11-08T19:32:05Z,1928,Arkaitz1974,\n== მოვლენები ==\n* 11 თებერვალი : შვეიცარიის...,g2uj2d76l6yrbtu1p4wvmnx7vacz64z,False,1928 მოვლენები 11 თებერვალი შვეიცარიის ქალაქ ...
282,823,2022-05-09T20:01:56Z,საქართველო,GiorgiXIII,"\n\nსაქართველო () — სახელმწიფო ევრაზიაში, კავკ...",9kmab2upb1nr0ad1ywgr6ldl787kyo3,False,საქართველო საქართველო სახელმწიფო ევრაზიაში კა...


In [12]:
''.join(sorted(list(set(''.join(kdf.title.values)))))

' !"$&\'()*+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz«\xad°±²º»½ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëíðñóôöúûüýĀāĂĄĆĈĊČčĎĐĒĔĖĘĚĜĞĠĢĤĦĨĪĬĮİıĴĶĹĻĽĿŁłŃńŅŇŊŌōŎŐŒŔŖŘŚśŜŞşŠšŢŤŦŨŪŬŮŰŲŴŶŸŹźŻżŽžƏǂə̧̣̀́̂̃̄̆̇̈̊̌̑̓ЀЁЂЃЄЅІЇЈЉЊЋЌЍЎЏАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяёѠѢѤѦѨѪѬѮѰѲѴѶѸѺѼѾҀҊҌҎҐҒҔҖҘҚҜҞҠҢҤҦҨҪҬҮҰҲҴҶҸҺҼҾӀӁӃӅӇӉӋӍӐӒӔӖӘәӚӜӞӠӢӤӦӨӪӬӮӰӲӴӶӸӺӼӾԀԂԄԆԈԊԌԎԐԒԔԖԘԚԜԞԠԢԤԦԨԪԬԮԱԲԳԴԵԶԷԸԹԺԻԼԽԾԿՀՁՂՃՄՅՆՇՈՉՊՋՌՍՎՏՐՑՒՓՔՕՖՠագդեէիլխհմյնոպվրւֆևֈ֍֎אاةتدرسصعلمويಂಏಕಗಡತನಪಯಳವಷಿೀು್Ⴭაბგდევზთიკლმნოპჟრსტუფქღყშჩცძწჭხჯჰჱჲჳჴჵჶჷჸჹჺ჻ჼჽჾჿ᠐᠑᠒᠓᠔᠕᠖᠗᠘᠙ᲅᲐᲑᲒᲖᲗᲘᲙᲛᲜᲞᲡᲢᲣᲥᲨᲩᲬ–—’“”„†•…′″₾№→−∞♯時樂用譜鄕ꙀꙂꙄꙆꙈꙊꙌꙎꙐꙒꙔꙖꙘꙚꙜꙞꙠꙢꙤꙦꙨꙪꙬꙮꚀꚂꚄꚆꚈꚊꚌꚎꚐꚒꚔꚖꚘꚚ보시악용향𐊠𐊡𐊢𐊣𐊤𐊥𐊦𐊧𐊨𐊩𐊪𐊫𐊬𐊭𐊮𐊯𐊰𐊱𐊲𐊳𐊴𐊵𐊶𐊷𐊸𐊹𐊺𐊻𐊼𐊽𐊾𐊿𐋀𐋁𐋂𐋃𐋄𐋅𐋆𐋇𐋈𐋉𐋊𐋋𐋌𐋍𐋎𐋏𐋐𐔰𐔱𐔲𐔳𐔴𐔵𐔶𐔷𐔸𐔹𐔺𐔻𐔼𐔽𐔾𐔿𐕀𐕁𐕂𐕃𐕄𐕅𐕆𐕇𐕈𐕉𐕊𐕋𐕌𐕍𐕎𐕏𐕐𐕑𐕒𐕓𐕔𐕕𐕖𐕗𐕘𐕙𐕚𐕛𐕜𐕝𐕞𐕟𐕠𐕡𐕢𐕣𐕯𐤀𐤁𐤂𐤃𐤄𐤅𐤆𐤇𐤈𐤉𐤊𐤋𐤌𐤍𐤎𐤏𐤐𐤑𐤒𐤓𐤔𐤕'

In [27]:
kdf[kdf.title.str.contains(':') & ~kdf.title.str.contains('კატეგორია') & ~kdf.title.str.contains('ფაილი')& ~kdf.title.str.contains('პორტალი')
& ~kdf.title.str.contains('მოდული')].sample(100)

,id,ts,title,author,text,sha1,isRedirection,cleantext
79044,125508,2021-12-25T12:28:58Z,UTC-04:00,InternetArchiveBot,"thumb|500px|UTC−04:00: ლურჯი (დეკემბერი), ნარი...",4awpjkk50aiw85hjw1dyi24370n40ln,False,04 00 500 04 00 ლურჯი დეკემბერი ნარინჯისფერი...
94824,153136,2021-09-27T19:03:49Z,ბნელი რაინდი: აღზევება,InternetArchiveBot,\n\nბნელი რაინდი: აღზევება () — რეჟისორ კრისტო...,1hso4tl55570ki69j595kztg0n9cj7i,False,ბნელი რაინდი აღზევება ბნელი რაინდი აღზევება რ...
46574,67137,2021-01-18T17:54:55Z,Advanced Dork:,InternetArchiveBot,thumb|right|Advanced Dork\n\nMozilla Firefox-ი...,qgwieh2k4vy2q39xfafl72p9y9j82pe,False,ის გაფართოება იძლევა სწრაფ კავშირს ის მოწინ...
203907,337699,2021-09-26T19:49:19Z,Wolfenstein: The New Order,InternetArchiveBot,\n\nWolfenstein: The New Order — მოქმედებითი-ს...,3z54w4f2khqhme1cpvlx152st4fg7du,False,მოქმედებითი სათავგადასავლო და პირველი პირის...
142032,231893,2022-01-03T14:12:38Z,რობინ ჰუდი: ქურდების უფლისწული,InternetArchiveBot,\n\n„რობინ ჰუდი“ () — 1991 წლის ამერიკული სათა...,n56u71vaw3uc8a4v7x3ebp0ai4uncir,False,რობინ ჰუდი ქურდების უფლისწული რობინ ჰუდი 1991...
207168,343974,2016-07-08T20:09:42Z,The Elder Scrolls V: Skyrim – Dragonborn,Gobrona,\n\nThe Elder Scrolls V: Skyrim – Dragonborn —...,n0zy48tpkjc0hru2d15n0kk24ot8r05,False,ღია სამყაროს მოქმედებითი როლური ვიდეო თამაშ...
99009,160946,2022-01-05T12:40:44Z,E.L.E. (Extinction Level Event): The Final Wor...,InternetArchiveBot,\n\n\n\nE.L.E. (Extinction Level Event): The F...,hxnw765140klkpvw2suz7w8ezruj5pd,False,ამერიკელი რეპერ ბასტა რაიმზის 1998 წლის სტუ...
167686,278602,2019-07-03T10:08:19Z,"ძიუდოს მსოფლიო ჩემპიონატი 2014: კაცები, 81 კგ",ჯეო,\n\n2014 წლის ძიუდოს მსოფლიო ჩემპიონატი კაცთა ...,obvxi4yke8louvzuhz9of7n8sik8lxn,False,ძიუდოს მსოფლიო ჩემპიონატი 2014 კაცები 81 კგ 2...
194877,321827,2016-02-18T07:13:12Z,მსოფლიო საფეხბურთო ჩემპიონატი 2006: ჯგუფი A,M.,მსოფლიო საფეხბურთო ჩემპიონატი 2006: ჯგუფი A — ...,cf10tbp7gtdw3ggb81puouv89tomohs,False,მსოფლიო საფეხბურთო ჩემპიონატი 2006 ჯგუფი მსოფ...
35941,50902,2021-08-12T11:38:25Z,დახმარება:მცირე რედაქტირება,Otogi,"\nმცირე შესწორების მონიშვნა მიუთითებს იმაზე, რ...",fvdpxpq3x8cnyim9y7ek4rcccgn153e,False,დახმარება მცირე რედაქტირება მცირე შესწორების ...


In [64]:
fulltext = '.'.join(kdf.cleantext.values)
sentences = fulltext.split('.')
sentences = [sentence.split(' ') for sentence in tqdm(sentences)]

for idx, sentence in tqdm(enumerate(sentences), total=len(sentences)):
    sentences[idx] = [
        token for token in sentence 
        if token != '' 
        and len(token) > 1
    ]

sentences = [sentence for sentence in tqdm(sentences) if sentence != []]

sentences[:5]

100%|██████████| 3362533/3362533 [00:00<00:00, 6412523.25it/s]


[['ედუარდ', 'შევარდნაძე'],
 ['ედუარდ', 'ამბროსის', 'ძე', 'შევარდნაძე'],
 ['იანვარი', 'სოფელი', 'მამათი', 'ოზურგეთის', 'მაზრა', 'საქართველოს', 'სსრ'],
 ['ივლისი',
  'თბილისი',
  'საქართველო',
  'ქართველი',
  'პოლიტიკოსი',
  'და',
  'სახელმწიფო',
  'მოღვაწე'],
 ['საქართველოს', 'მეორე', 'პრეზიდენტი']]

In [74]:
wordfreq = defaultdict(int)

for sentence in tqdm(sentences):
    for token in sentence:
        wordfreq[token] += 1

print(f"Total words:\t\t\t{len(wordfreq)}")
print(f"Total words with count=1:\t{len(list(filter(lambda x : x[1] == 1, wordfreq.items())))}")

100%|██████████| 2754036/2754036 [00:06<00:00, 446409.97it/s]


Total words:			1079347
Total words with count=1:	571881


In [76]:
# Filter out spelling errors and extremely infrequent words that models are unlikely to capture any useful signals
tokenCorpus = [
    [token for token in sentence if wordfreq[token] > 1]
    for sentence in tqdm(sentences)
]

# Filter out single-word sentences
tokenCorpus = [sentence for sentence in tqdm(tokenCorpus) if len(sentence) > 1]

100%|██████████| 2754036/2754036 [00:00<00:00, 5939979.46it/s]


In [77]:
with open("kawiki_corpus_sentences.txt", "w") as file:
    for doc in tqdm(tokenCorpus):
        file.write(' '.join(doc) + '\n')

100%|██████████| 2562447/2562447 [00:02<00:00, 1000135.00it/s]


100%|██████████| 2611926/2611926 [00:02<00:00, 928658.19it/s]


# Article-level Tokenization

In [78]:
kdf = pd.read_csv("../v1/kawiki.csv")
kdf.text = kdf.text.fillna('')

kdf.head(3)

,id,ts,title,author,text,sha1,isRedirection
0,2,2004-01-26T08:48:14Z,ვიკიპედია:Orphaned articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",ptlg4dsza52o10rnmk59f282y0fptzi,False
1,3,2004-01-26T00:26:51Z,ვიკიპედია:Most wanted articles,204.95.98.251,,pezx7385o3910tfwsxb3suac2dw5g9i,False
2,4,2004-01-26T00:38:52Z,ვიკიპედია:Short articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",84gitu5sgfaec85yfccjzvl2kfafvn1,False


In [79]:
def cleanup(doc):
    # doc = re.sub(r"[^Ⴀ-ჿⴀ-ⴥᲐ-Ჿ0-9-–—_%a-zA-Z]", ' ', doc)
    doc = str(doc).lower()
    doc = re.sub(r"[^Ⴀ-ჿⴀ-ⴥᲐ-Ჿ]", ' ', doc)
    doc = re.sub(r"\s+", ' ', doc)
    return doc


kdf['cleantext'] = kdf.text.parallel_apply(cleanup)
kdf.head(3)

,id,ts,title,author,text,sha1,isRedirection,cleantext
0,2,2004-01-26T08:48:14Z,ვიკიპედია:Orphaned articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",ptlg4dsza52o10rnmk59f282y0fptzi,False,
1,3,2004-01-26T00:26:51Z,ვიკიპედია:Most wanted articles,204.95.98.251,,pezx7385o3910tfwsxb3suac2dw5g9i,False,
2,4,2004-01-26T00:38:52Z,ვიკიპედია:Short articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",84gitu5sgfaec85yfccjzvl2kfafvn1,False,


In [80]:
# Drop useless wiki articles

# Empty articles
kdf.drop(kdf[(kdf.cleantext == ' ') | (kdf.cleantext == '')].index, inplace=True)
kdf.drop(kdf[kdf.cleantext.apply(lambda x: bool(re.compile("[ა-ჿ]").search(x)))==False].index, inplace=True)

# Mediawiki articles
kdf.drop(kdf[kdf.title.str.startswith("მედიავიკი:")].index, inplace=True)
kdf.drop(kdf[kdf.title.str.startswith("ვიკიპედია:")].index, inplace=True)
kdf.drop(kdf[kdf.title.str.startswith("თარგი:")].index, inplace=True)

# Redirection
kdf.drop(kdf[kdf.isRedirection].index, inplace=True)

kdf

,id,ts,title,author,text,sha1,isRedirection,cleantext
234,604,2022-04-15T07:50:59Z,ედუარდ შევარდნაძე,Melberg,\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი...,k4tckhizmxggllhid4m0uob0nmwbu0h,False,ედუარდ ამბროსის ძე შევარდნაძე დ იანვარი სოფელ...
236,608,2020-12-01T07:15:40Z,1 იანვარი,Namdvili,\n1 იანვარი — გრიგორიანული კალენდრის პირველი დ...,9hua4ulox6vj429j2nbx3404fylhw5t,False,იანვარი გრიგორიანული კალენდრის პირველი დღე წლ...
237,609,2020-01-18T07:59:48Z,25 იანვარი,95.104.15.233,\n25 იანვარი — გრიგორიანული კალენდრის 25-ე დღე...,nburza2tu9lqypz9cirawwpcg3s7e8z,False,იანვარი გრიგორიანული კალენდრის ე დღე წლის ბოლ...
238,610,2021-11-08T19:32:05Z,1928,Arkaitz1974,\n== მოვლენები ==\n* 11 თებერვალი : შვეიცარიის...,g2uj2d76l6yrbtu1p4wvmnx7vacz64z,False,მოვლენები თებერვალი შვეიცარიის ქალაქ ზანქტ მო...
282,823,2022-05-09T20:01:56Z,საქართველო,GiorgiXIII,"\n\nსაქართველო () — სახელმწიფო ევრაზიაში, კავკ...",9kmab2upb1nr0ad1ywgr6ldl787kyo3,False,საქართველო სახელმწიფო ევრაზიაში კავკასიაში შა...
...,...,...,...,...,...,...,...,...
320738,532936,2022-05-19T20:24:07Z,კატეგორია:სამხრეთ კორეის საერთაშორისო ხელშეკრუ...,იაკობ მახარაძე,კატეგორია:საერთაშორისო ხელშეკრულებები ქვეყნები...,1mvdnl2ggxg80q12sh5aw4dwfbbt3pu,False,კატეგორია საერთაშორისო ხელშეკრულებები ქვეყნები...
320740,532938,2022-05-20T04:35:44Z,ვახტანგ რობაქიძე,Jaba1977,ვახტანგ რობაქიძე (მოსამართლეობის კანდიდატი) და...,iavvdscaounfaz6sqowhc2ogsvp4u0l,False,ვახტანგ რობაქიძე მოსამართლეობის კანდიდატი დაბ
320741,532939,2022-05-20T08:54:31Z,ნინა კარპაჩოვა,Ekkatterrinna,\n\nნინა კარპაჩოვა (უკრ. Ні́на Іва́нівна Карпа...,m955153juz99zg6n3e4pphtna8s6f8t,False,ნინა კარპაჩოვა უკრ დ აგვისტო მოლდავეთის სსრ უ...
320743,532947,2022-05-20T08:27:24Z,ჰაილბრონის აჩრდილი,Lisztomaniac,\nthumb|right|250px|მემორიალური ნიშნული ოფიცერ...,6sgtfdyggb9svx8whe22owjhwkm8fia,False,მემორიალური ნიშნული ოფიცერი კიზევეტერის მკვლე...


In [81]:
kdf['cleantext'] = kdf['title'].apply(cleanup) + ' ' + kdf['cleantext'] + ' ' + kdf['title'].apply(cleanup)
kdf.head()

,id,ts,title,author,text,sha1,isRedirection,cleantext
234,604,2022-04-15T07:50:59Z,ედუარდ შევარდნაძე,Melberg,\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი...,k4tckhizmxggllhid4m0uob0nmwbu0h,False,ედუარდ შევარდნაძე ედუარდ ამბროსის ძე შევარდნა...
236,608,2020-12-01T07:15:40Z,1 იანვარი,Namdvili,\n1 იანვარი — გრიგორიანული კალენდრის პირველი დ...,9hua4ulox6vj429j2nbx3404fylhw5t,False,იანვარი იანვარი გრიგორიანული კალენდრის პირვე...
237,609,2020-01-18T07:59:48Z,25 იანვარი,95.104.15.233,\n25 იანვარი — გრიგორიანული კალენდრის 25-ე დღე...,nburza2tu9lqypz9cirawwpcg3s7e8z,False,იანვარი იანვარი გრიგორიანული კალენდრის ე დღე...
238,610,2021-11-08T19:32:05Z,1928,Arkaitz1974,\n== მოვლენები ==\n* 11 თებერვალი : შვეიცარიის...,g2uj2d76l6yrbtu1p4wvmnx7vacz64z,False,მოვლენები თებერვალი შვეიცარიის ქალაქ ზანქტ ...
282,823,2022-05-09T20:01:56Z,საქართველო,GiorgiXIII,"\n\nსაქართველო () — სახელმწიფო ევრაზიაში, კავკ...",9kmab2upb1nr0ad1ywgr6ldl787kyo3,False,საქართველო საქართველო სახელმწიფო ევრაზიაში კა...


In [91]:
articles = kdf.cleantext.values
articles = [sentence.split(' ') for sentence in tqdm(articles)]

for idx, sentence in tqdm(enumerate(articles), total=len(articles)):
    articles[idx] = [
        token for token in sentence 
        if token != '' 
        and len(token) > 1
    ]

articles = [sentence for sentence in tqdm(articles) if sentence != []]

articles[0][:10]

100%|██████████| 229121/229121 [00:00<00:00, 5186565.31it/s]


['ედუარდ',
 'შევარდნაძე',
 'ედუარდ',
 'ამბროსის',
 'ძე',
 'შევარდნაძე',
 'იანვარი',
 'სოფელი',
 'მამათი',
 'ოზურგეთის']

In [93]:
wordfreq = defaultdict(int)

for article in tqdm(articles):
    for token in article:
        wordfreq[token] += 1

print(f"Total words:\t\t\t{len(wordfreq)}")
print(f"Total words with count=1:\t{len(list(filter(lambda x : x[1] == 1, wordfreq.items())))}")

100%|██████████| 229121/229121 [00:06<00:00, 32737.60it/s]


Total words:			1079347
Total words with count=1:	570816


In [94]:
# Filter out spelling errors and extremely infrequent words that models are unlikely to capture any useful signals
tokenCorpus = [
    [token for token in article if wordfreq[token] > 1]
    for article in tqdm(articles)
]

# Filter out single-word articles
tokenCorpus = [article for article in tqdm(tokenCorpus) if len(article) > 1]

100%|██████████| 229121/229121 [00:00<00:00, 3471970.08it/s]


In [95]:
with open("kawiki_corpus_articles.txt", "w") as file:
    for doc in tqdm(tokenCorpus):
        file.write(' '.join(doc) + '\n')

100%|██████████| 229120/229120 [00:01<00:00, 137328.62it/s]
